In [1]:
#@title 安裝相關套件 (yt-dlp, openAI API, Pydub)
! pip install --upgrade pip
! pip install yt-dlp
! pip install openai
! pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 18.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.5 MB/s eta 0:00:00


In [2]:
#@title 下載 youtube 影片 
import yt_dlp

#@markdown ### 輸出檔案名稱 (如輸入 audio 會產生 audio.mp3):
filename = 'audio' #@param {type:"string"} 

#@markdown ### Youtube 連結:
url = 'https://www.youtube.com/watch?v=PRp3kJtMQwU' #@param {type:"string"} 


# 設定選項
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': filename , 
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

# 建立 yt_dlp 下載器物件
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=PRp3kJtMQwU
[youtube] PRp3kJtMQwU: Downloading webpage
[youtube] PRp3kJtMQwU: Downloading android player API JSON
[youtube] PRp3kJtMQwU: Downloading player 21246a91
[info] PRp3kJtMQwU: Downloading 1 format(s): 251
[dashsegments] Total fragments: 3
[download] Destination: audio
[download] 100% of   22.96MiB in 00:00:01 at 14.67MiB/s              
[ExtractAudio] Destination: audio.mp3
Deleting original file audio (pass -k to keep)


In [3]:
#@title 分割 youtube 影片 
from pydub import AudioSegment

#@markdown ### 分割檔案的長度（單位：毫秒）:
segment_length = 1000000 #@param {type:"integer"}

# 載入 MP3 音檔
sound = AudioSegment.from_file(f'{filename}.mp3', format='mp3')

sound_track = []
# 將音檔分割成多個檔案
for i, chunk in enumerate(sound[::segment_length]):
    # 設定分割檔案的檔名
    chunk.export(f'output_{i}.mp3', format='mp3')
    audio_file = open(f'output_{i}.mp3', "rb")
    sound_track.append(audio_file)
    


In [5]:
import openai
#@markdown ### 填入 OpenAI API Secret Key:
openai.api_key = '' #@param {type:"string"}
openai.api_key = TOKEN

transcripts_ary = []
for audio_file in sound_track:
  transcript = openai.Audio.transcribe(
      "whisper-1", 
      audio_file, 
      response_format = 'srt',
      prompt = "這是一段關於吳淡如與Joeman 討論台灣人是否應該在日本買房的訪談")

  transcripts_ary.append(transcript)

In [ ]:
print(transcripts_ary[0])

In [ ]:
print(transcripts_ary[1])

In [9]:
! pip install pysrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=cfe4002aa8c383c6739de04d5f8c3b25dd9c74deabd7442ac868bb4d28d7b254
  Stored in directory: /root/.cache/pip/wheels/c3/34/f1/ae1d86b7f454100c10f7ab8dc411303b7834e7f40e343ca2c0
Successfully built pysrt


In [15]:
import pysrt

# Read the first SRT file
subtitles1 = pysrt.from_string(transcripts_ary[0])

# Read the second SRT file
subtitles2 = pysrt.from_string(transcripts_ary[1])



In [ ]:
subtitles1

In [16]:
max_time = pysrt.SubRipTime(seconds = 1000)
for sub in subtitles1:
  sub.start = sub.start if sub.start < max_time else max_time
  sub.end = sub.end if sub.end < max_time else max_time

In [17]:
shift_time = pysrt.SubRipTime(seconds = 1000)
for sub in subtitles2:
  sub.start = sub.start + shift_time
  sub.end = sub.end + shift_time

In [18]:
subtitles1.extend(subtitles2)

In [19]:
subtitles1.save('merged_subtitles.srt')